## Data Cleaning

Now since we have extracted data from website, it is not cleaned and ready to be analyzed yet. The reviews section will be cleaned for punctuations, spellings and other characters.

In [1]:
#imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

#regex
import re

In [3]:
#create a dataframe from csv file

cwd=os.getcwd()
df=pd.read_csv(cwd+"/BA-reviews.csv", index_col=0)

In [5]:
df.head()

,reviews,stars,date,country
0,Not Verified | I flew internationally for the...,5.0,5th August 2024,United States
1,"✅ Trip Verified | BA, after subsequent delays ...",1.0,3rd August 2024,Italy
2,Not Verified | It is embarrassing to have th...,1.0,2nd August 2024,Australia
3,✅ Trip Verified | Flight cancelled due to bad ...,1.0,2nd August 2024,United Kingdom
4,✅ Trip Verified | British Airways oversold my...,10.0,30th July 2024,United States


We will also create a column which mention if the user is verified or not.

In [7]:
df['verified']=df.reviews.str.contains("Trip Verified")

In [9]:
df['verified']

0       False
1        True
2       False
3        True
4        True
        ...  
3495    False
3496    False
3497    False
3498    False
3499    False
Name: verified, Length: 3500, dtype: bool

### Cleaning Reviews

We will extract the column of reviews into a separate dataframe and clean it for semantic analysis

In [11]:
#for lemmatization of words we will use nltk library
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemma=WordNetLemmatizer()

reviews_data=df.reviews.str.strip("✅ Trip Verified |")

#create an empty list to collect cleaned data corpus
corpus=[]

#loop through each review, remove punctuations, small cases, join and add it to corpus
for rev in reviews_data:
    rev=re.sub('[^a-zA-Z]',' ',rev)
    rev=rev.lower()
    rev=rev.split()
    rev=[lemma.lemmatize(word) for word in rev if not word in stopwords.words('english')]
    rev=" ".join(rev)
    corpus.append(rev)

In [13]:
#add the corpus to the original dataframe

df['corpus']=corpus

In [15]:
df.head()

,reviews,stars,date,country,verified,corpus
0,Not Verified | I flew internationally for the...,5.0,5th August 2024,United States,False,verified flew internationally first time life ...
1,"✅ Trip Verified | BA, after subsequent delays ...",1.0,3rd August 2024,Italy,True,ba subsequent delay postponed departure time i...
2,Not Verified | It is embarrassing to have th...,1.0,2nd August 2024,Australia,False,verified embarrassing word british attached ai...
3,✅ Trip Verified | Flight cancelled due to bad ...,1.0,2nd August 2024,United Kingdom,True,flight cancelled due bad weather ba contacted ...
4,✅ Trip Verified | British Airways oversold my...,10.0,30th July 2024,United States,True,british airway oversold lhr lax flight booked ...


### Cleaning/Format date

In [14]:
df.dtypes

reviews      object
stars       float64
date         object
country      object
verified       bool
corpus       object
dtype: object

In [15]:
df.date

0         5th August 2024
1         3rd August 2024
2         2nd August 2024
3         2nd August 2024
4          30th July 2024
              ...        
3495    13th October 2014
3496    13th October 2014
3497    13th October 2014
3498    12th October 2014
3499    12th October 2014
Name: date, Length: 3500, dtype: object

In [17]:
#convert the date to datetime format

# Create  DataFrame 
data = {'date': ['5th August 2024', '3rd August 2024', '2nd August 2024', '2nd August 2024', '30th July 2024', ...]}

df1 = pd.DataFrame(data)

# Remove ordinal indicators
df['dates'] = df['date'].str.replace(r'\b(\d+)(st|nd|rd|th)\b', r'\1', regex=True)

# Convert the 'date' column to datetime
df['dates'] = pd.to_datetime(df['dates'], format='%d %B %Y')

print(df['dates'])



0      2024-08-05
1      2024-08-03
2      2024-08-02
3      2024-08-02
4      2024-07-30
          ...    
3495   2014-10-13
3496   2014-10-13
3497   2014-10-13
3498   2014-10-12
3499   2014-10-12
Name: dates, Length: 3500, dtype: datetime64[ns]


In [17]:
df['dates'].head()

0   2024-08-05
1   2024-08-03
2   2024-08-02
3   2024-08-02
4   2024-07-30
Name: dates, dtype: datetime64[ns]

In [19]:
df1=df.drop('date', axis=1)

In [199]:
df1.head()

,reviews,stars,country,verified,corpus,dates
0,Not Verified | I flew internationally for the...,5.0,United States,False,verified flew internationally first time life ...,2024-08-05
1,"✅ Trip Verified | BA, after subsequent delays ...",1.0,Italy,True,ba subsequent delay postponed departure time i...,2024-08-03
2,Not Verified | It is embarrassing to have th...,1.0,Australia,False,verified embarrassing word british attached ai...,2024-08-02
3,✅ Trip Verified | Flight cancelled due to bad ...,1.0,United Kingdom,True,flight cancelled due bad weather ba contacted ...,2024-08-02
4,✅ Trip Verified | British Airways oversold my...,10.0,United States,True,british airway oversold lhr lax flight booked ...,2024-07-30


## Check for null values

In [67]:
df1.isnull().sum()

reviews     0
stars       3
country     2
verified    0
corpus      0
dates       0
dtype: int64

We have 3 missing values in stars and 2 missing values in country. We will drop those from dataframe

In [21]:
# Remove the 'stars' and 'country' columns
df1.dropna(subset=['stars', 'country'], inplace=True)


In [19]:
df1.shape

(3495, 6)

In [21]:
df1.isnull().sum()

reviews     0
stars       0
country     0
verified    0
corpus      0
dates       0
dtype: int64

In [249]:
df1.stars.unique()

array([ 5.,  1., 10.,  3.,  6.,  9.,  2.,  8.,  7.,  4.])

In [23]:
#change the stars from float to string
df1['stars'] = np.round(df1['stars']).astype(int).astype(str)

In [25]:
df1.dtypes

reviews             object
stars               object
country             object
verified              bool
corpus              object
dates       datetime64[ns]
dtype: object

In [25]:
#Resetting the index
df1.reset_index(drop=True, inplace=True)
df=df1.copy()
df.head()

,reviews,stars,country,verified,corpus,dates
0,Not Verified | I flew internationally for the...,5,United States,False,verified flew internationally first time life ...,2024-08-05
1,"✅ Trip Verified | BA, after subsequent delays ...",1,Italy,True,ba subsequent delay postponed departure time i...,2024-08-03
2,Not Verified | It is embarrassing to have th...,1,Australia,False,verified embarrassing word british attached ai...,2024-08-02
3,✅ Trip Verified | Flight cancelled due to bad ...,1,United Kingdom,True,flight cancelled due bad weather ba contacted ...,2024-08-02
4,✅ Trip Verified | British Airways oversold my...,10,United States,True,british airway oversold lhr lax flight booked ...,2024-07-30


Now our data is all cleaned and ready for data visualization and data analysis

In [75]:
# export the cleaned data

df.to_csv(cwd + "/cleaned-BA-reviews.csv")